In [14]:
import pickle
import pandas as pd
import numpy as np

In [15]:
with open('dangjin_raw.pkl', 'rb') as f:
    dangjin = pickle.load(f)
with open('ulsan_raw.pkl', 'rb') as f:
    ulsan = pickle.load(f)

In [16]:
# cloud
dangjin_convert_cloud, ulsan_convert_cloud = ({1.0: 2.7635372029606544,
  2.0: 3.8820678513731823,
  3.0: 6.18494516450648,
  4.0: 7.961345381526105},
 {2.0: 3.5910064239828694,
  1.0: 1.721059516023545,
  3.0: 6.145117540687161,
  4.0: 8.638197424892704}) # from advanced_preprocessing.ipynb

dangjin['Cloud'].replace(dangjin_convert_cloud, inplace=True)
ulsan['Cloud'].replace(ulsan_convert_cloud, inplace=True)

In [17]:
# wind
def preprocess_wind(data):
    '''
    data: pd.DataFrmae which contains the columns 'WindSpeed' and 'WindDirection'
    '''

    # degree to radian
    wind_direction_radian = data['WindDirection'] * np.pi / 180

    # polar coordinate to cartesian coordinate
    wind_x = data['WindSpeed'] * np.cos(wind_direction_radian)
    wind_y = data['WindDirection'] * np.sin(wind_direction_radian)

    # name pd.series
    wind_x.name = 'Wind_X'
    wind_y.name = 'Wind_Y'

    return wind_x, wind_y

dangjin = dangjin.join(preprocess_wind(dangjin))
dangjin.drop(columns=['WindDirection','WindSpeed'], inplace=True)

ulsan = ulsan.join(preprocess_wind(ulsan))
ulsan.drop(columns=['WindDirection','WindSpeed'], inplace=True)

In [18]:
# interpolate
dangjin.interpolate(method='quadratic', inplace=True)
ulsan.interpolate(method='quadratic', inplace=True)

In [19]:
# save
with open('dangjin_final.pkl', 'wb') as f:
    pickle.dump(dangjin.iloc[1:], f)
with open('ulsan_final.pkl', 'wb') as f:
    pickle.dump(ulsan.iloc[1:],f)